In [54]:
import json
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import word_tokenize
nltk.download('maxent_ne_chunker')
nltk.download('words')
from nltk.tag import pos_tag
from nltk.chunk import ne_chunk
from bs4 import BeautifulSoup
import requests
import re
import operator

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/jonathandai/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/jonathandai/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/jonathandai/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /Users/jonathandai/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [33]:
with open('gg2020.json', encoding='utf8') as f:
    lines = f.readlines()

gg_data = []
for line in lines:
    gg_data.append(json.loads(line))
    

In [ ]:
from pandas import DataFrame
df = DataFrame.from_csv("tsv.tsv", sep="\t")

In [52]:
tweets_with_host = []
for tweet in gg_data: 
    tweet_text = tweet['text']
    if("host" in tweet_text.lower()) :
        tweets_with_host.append(tweet)
len(tweets_with_host)

5146

In [59]:
def get_names(text):
    article = nlp(text)
    labels = [x.label_ for x in article.ents]
    [(x.orth_,x.pos_, x.lemma_) for x in [y 
                                      for y
                                      in nlp(text) 
                                      if not y.is_stop and y.pos_ != 'PUNCT']]
    parts_of_speech = dict([(str(x), x.label_) for x in nlp(text).ents])
    names = []
    for (key, value) in parts_of_speech.items() :
        if(value == 'PERSON') :
            names.append(key)
    return names 

In [64]:
def get_host_name():
    host_name_count = {}
    for tweet in tweets_with_host: 
        names = get_names(tweet["text"])
        for name in names :
            if name in host_name_count :
                host_name_count[name] = host_name_count[name] + 1
            else :
                host_name_count[name] = 1
    host = max(host_name_count.items(), key=operator.itemgetter(1))[0]
    print(host)
    return host
get_host_name()

Ricky Gervais


'Ricky Gervais'

In [68]:
text = gg_data[610]['text']
article = nlp(text)
labels = [x.label_ for x in article.ents]
[(x.orth_,x.pos_, x.lemma_) for x in [y 
                                  for y
                                  in nlp(text) 
                                  if not y.is_stop and y.pos_ != 'PUNCT']]
parts_of_speech = dict([(str(x), x.label_) for x in nlp(text).ents])
print(parts_of_speech)

{'Awkwafina': 'PERSON', 'last night': 'TIME', 'Farewell': 'ORG'}


In [35]:
ne_tree = ne_chunk(pos_tag(word_tokenize(gg_data[0]["text"])))
print(ne_tree)

(S
  •/JJ
  (ORGANIZATION Priyanka/NNP Chopra/NNP)
  and/CC
  (PERSON Nick/NNP Jonas/NNP)
  at/IN
  the/DT
  77th/CD
  (PERSON Annual/NNP Golden/NNP Globe/NNP Awards/NNP)
  present/VBD
  the/DT
  award/NN
  for/IN
  (GPE Best/NNP)
  TV/NN
  series/NN
  for/IN
  a/DT
  musical/JJ
  or/CC
  comedy/NN
  •/JJ
  #/#
  goldenglobes2020/JJ
  @/NNP
  (PERSON Golden/NNP Globes/NNP)
  https/NN
  :/:
  //t.co/fivnKsE9Ib/NN)


In [36]:
def preprocess(sent):
    sent = nltk.word_tokenize(sent)
    sent = nltk.pos_tag(sent)
    return sent

In [37]:
sent = preprocess(gg_data[0]["text"])
sent

[('•', 'JJ'),
 ('Priyanka', 'NNP'),
 ('Chopra', 'NNP'),
 ('and', 'CC'),
 ('Nick', 'NNP'),
 ('Jonas', 'NNP'),
 ('at', 'IN'),
 ('the', 'DT'),
 ('77th', 'CD'),
 ('Annual', 'NNP'),
 ('Golden', 'NNP'),
 ('Globe', 'NNP'),
 ('Awards', 'NNP'),
 ('present', 'VBD'),
 ('the', 'DT'),
 ('award', 'NN'),
 ('for', 'IN'),
 ('Best', 'NNP'),
 ('TV', 'NN'),
 ('series', 'NN'),
 ('for', 'IN'),
 ('a', 'DT'),
 ('musical', 'JJ'),
 ('or', 'CC'),
 ('comedy', 'NN'),
 ('•', 'JJ'),
 ('#', '#'),
 ('goldenglobes2020', 'JJ'),
 ('@', 'NNP'),
 ('Golden', 'NNP'),
 ('Globes', 'NNP'),
 ('https', 'NN'),
 (':', ':'),
 ('//t.co/fivnKsE9Ib', 'NN')]

In [38]:
pattern = 'NP: {<DT>?<JJ>*<NN>}'

In [39]:
cp = nltk.RegexpParser(pattern)
cs = cp.parse(sent)
print(cs)

(S
  •/JJ
  Priyanka/NNP
  Chopra/NNP
  and/CC
  Nick/NNP
  Jonas/NNP
  at/IN
  the/DT
  77th/CD
  Annual/NNP
  Golden/NNP
  Globe/NNP
  Awards/NNP
  present/VBD
  (NP the/DT award/NN)
  for/IN
  Best/NNP
  (NP TV/NN)
  (NP series/NN)
  for/IN
  a/DT
  musical/JJ
  or/CC
  (NP comedy/NN)
  •/JJ
  #/#
  goldenglobes2020/JJ
  @/NNP
  Golden/NNP
  Globes/NNP
  (NP https/NN)
  :/:
  (NP //t.co/fivnKsE9Ib/NN))


In [41]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

In [42]:
doc = nlp(gg_data[0]["text"])
pprint([(X.text, X.label_) for X in doc.ents])

[('Priyanka Chopra', 'ORG'),
 ('Nick Jonas', 'PERSON'),
 ('77th', 'ORDINAL'),
 ('Annual Golden Globe Awards', 'EVENT'),
 ('goldenglobes2020', 'MONEY')]


In [43]:
pprint([(X, X.ent_iob_, X.ent_type_) for X in doc])

[(•, 'O', ''),
 (Priyanka, 'B', 'ORG'),
 (Chopra, 'I', 'ORG'),
 (and, 'O', ''),
 (Nick, 'B', 'PERSON'),
 (Jonas, 'I', 'PERSON'),
 (at, 'O', ''),
 (the, 'O', ''),
 (77th, 'B', 'ORDINAL'),
 (Annual, 'B', 'EVENT'),
 (Golden, 'I', 'EVENT'),
 (Globe, 'I', 'EVENT'),
 (Awards, 'I', 'EVENT'),
 (present, 'O', ''),
 (the, 'O', ''),
 (award, 'O', ''),
 (for, 'O', ''),
 (Best, 'O', ''),
 (TV, 'O', ''),
 (series, 'O', ''),
 (for, 'O', ''),
 (a, 'O', ''),
 (musical, 'O', ''),
 (or, 'O', ''),
 (comedy, 'O', ''),
 (•, 'O', ''),
 (#, 'O', ''),
 (goldenglobes2020, 'B', 'MONEY'),
 (@, 'O', ''),
 (Golden, 'O', ''),
 (Globes, 'O', ''),
 (https://t.co/fivnKsE9Ib, 'O', '')]


In [46]:
from bs4 import BeautifulSoup
import requests
import re

In [47]:
def url_to_string(url):
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html, 'html5lib')
    for script in soup(["script", "style", 'aside']):
        script.extract()
    return " ".join(re.split(r'[\n\t]+', soup.get_text()))

In [48]:
first_tweet = gg_data[0]["text"]
article = nlp(first_tweet)

In [49]:
labels = [x.label_ for x in article.ents]
Counter(labels)

Counter({'ORG': 1, 'PERSON': 1, 'ORDINAL': 1, 'EVENT': 1, 'MONEY': 1})

In [50]:
[(x.orth_,x.pos_, x.lemma_) for x in [y 
                                      for y
                                      in nlp(first_tweet) 
                                      if not y.is_stop and y.pos_ != 'PUNCT']]

[('Priyanka', 'PROPN', 'Priyanka'),
 ('Chopra', 'PROPN', 'Chopra'),
 ('Nick', 'PROPN', 'Nick'),
 ('Jonas', 'PROPN', 'Jonas'),
 ('77th', 'ADJ', '77th'),
 ('Annual', 'PROPN', 'Annual'),
 ('Golden', 'PROPN', 'Golden'),
 ('Globe', 'PROPN', 'Globe'),
 ('Awards', 'PROPN', 'Awards'),
 ('present', 'VERB', 'present'),
 ('award', 'NOUN', 'award'),
 ('Best', 'ADJ', 'good'),
 ('TV', 'NOUN', 'tv'),
 ('series', 'NOUN', 'series'),
 ('musical', 'ADJ', 'musical'),
 ('comedy', 'NOUN', 'comedy'),
 ('•', 'X', '•'),
 ('#', 'NOUN', '#'),
 ('goldenglobes2020', 'NOUN', 'goldenglobes2020'),
 ('@', 'ADP', '@'),
 ('Golden', 'PROPN', 'Golden'),
 ('Globes', 'PROPN', 'Globes'),
 ('https://t.co/fivnKsE9Ib', 'X', 'https://t.co/fivnkse9ib')]

In [51]:
dict([(str(x), x.label_) for x in nlp(first_tweet).ents])

{'Priyanka Chopra': 'ORG',
 'Nick Jonas': 'PERSON',
 '77th': 'ORDINAL',
 'Annual Golden Globe Awards': 'EVENT',
 'goldenglobes2020': 'MONEY'}